## Face Feathring Image Preprocessing ##

- male/feamale annotaion json file is the output of manual segmentation in COCO format.
- The code below reads the annotation file and generates a new image with mask applied to the original image.
- Then, the masked image is feathered to create a new image with the face feathering effect.


### Masking Male/Female

In [1]:
import json
import numpy as np
import cv2
import os
from collections import defaultdict

# Load JSON
with open("male_annotations.json", "r") as f:
	coco = json.load(f)

images = coco['images']
annotations = coco['annotations']

ann_by_image = defaultdict(list)
for ann in annotations:
	ann_by_image[ann['image_id']].append(ann)

for img_info in images:
	file_name = img_info["file_name"].split(".")[0]
	file_name = ".".join(file_name.rsplit("_",1))
	img_id = img_info["id"]
	h, w = img_info["height"], img_info["width"]

	img_path = os.path.join("male", file_name)
	image = cv2.imread(img_path)

	# Create blank mask
	mask = np.zeros((h, w), dtype=np.uint8)

	for ann in ann_by_image[img_id]:
		segmentation = ann['segmentation'][0]  # only one polygon expected
		pts = np.array(segmentation, dtype=np.int32).reshape((-1, 2))
		cv2.fillPoly(mask, [pts], color=255)  # fill polygon as white
	mask = cv2.resize(mask, (w, h)).astype(np.uint8)
	white_bg = np.ones_like(image, dtype=np.uint8) * 255

	# Convert mask to 3 channels
	mask_3ch = cv2.merge([mask, mask, mask])

	# Combine masked image with white background
	masked_image = np.where(mask_3ch == 255, image, white_bg)
	# save the masked image
	output_path = os.path.join("masked_male", f"{file_name}.png")
	if not os.path.exists("masked_male"):
		os.makedirs("masked_male")
	cv2.imwrite(output_path, masked_image)

In [2]:
# Load JSON
with open("female_annotations.json", "r") as f:
	coco = json.load(f)

images = coco['images']
annotations = coco['annotations']

ann_by_image = defaultdict(list)
for ann in annotations:
	ann_by_image[ann['image_id']].append(ann)

for img_info in images:
	file_name = img_info["file_name"].split(".")[0]
	file_name = ".".join(file_name.rsplit("_",1))
	img_id = img_info["id"]
	h, w = img_info["height"], img_info["width"]

	img_path = os.path.join("female", file_name)
	image = cv2.imread(img_path)

	# Create blank mask
	mask = np.zeros((h, w), dtype=np.uint8)

	for ann in ann_by_image[img_id]:
		segmentation = ann['segmentation'][0]  # only one polygon expected
		pts = np.array(segmentation, dtype=np.int32).reshape((-1, 2))
		cv2.fillPoly(mask, [pts], color=255)  # fill polygon as white
	mask = cv2.resize(mask, (w, h)).astype(np.uint8)
	white_bg = np.ones_like(image, dtype=np.uint8) * 255

	# Convert mask to 3 channels
	mask_3ch = cv2.merge([mask, mask, mask])

	# Combine masked image with white background
	masked_image = np.where(mask_3ch == 255, image, white_bg)
	# save the masked image
	output_path = os.path.join("masked_female", f"{file_name}.png")
	if not os.path.exists("masked_female"):
		os.makedirs("masked_female")
	cv2.imwrite(output_path, masked_image)

In [3]:
import json
import numpy as np
import cv2
import os
from collections import defaultdict

with open("male_annotations.json", "r") as f:
	coco = json.load(f)

images = coco['images']
annotations = coco['annotations']

ann_by_image = defaultdict(list)
for ann in annotations:
	ann_by_image[ann['image_id']].append(ann)

# Create output directory if it doesn't exist
output_dir = "masked_male_feathered"
os.makedirs(output_dir, exist_ok=True)


for img_info in images:
	file_name = img_info["file_name"].split(".")[0]
	file_name = ".".join(file_name.rsplit("_",1))
	img_id = img_info["id"]
	h, w = img_info["height"], img_info["width"]

	# Construct the image path correctly from the original file_name
	img_path = os.path.join("male", file_name)
	image = cv2.imread(img_path)

	if image is None:
		print(f"Warning: Could not read image at {img_path}")
		continue

	# Create blank mask
	mask = np.zeros((h, w), dtype=np.uint8)

	for ann in ann_by_image[img_id]:
		# Check if segmentation exists and is not empty
		if 'segmentation' in ann and ann['segmentation']:
			segmentation = ann['segmentation'][0]
			if segmentation: # Ensure the polygon is not empty
				pts = np.array(segmentation, dtype=np.int32).reshape((-1, 2))
				cv2.fillPoly(mask, [pts], color=255)

	# Apply a Gaussian blur to the binary mask. This softens the edges,
	# turning them from sharp white to a smooth gray gradient.
	# You can adjust the kernel size (e.g., (21, 21)) to increase/decrease the feathering amount.
	# The kernel size must be a pair of odd numbers.
	blur_kernel_size = (21, 21)
	blurred_mask = cv2.GaussianBlur(mask, blur_kernel_size, 0)

	# Normalize the blurred mask to be float values between 0.0 and 1.0.
	# This will act as our alpha channel.
	alpha = blurred_mask.astype(float) / 255.0
	alpha_3ch = cv2.merge([alpha, alpha, alpha])

	# Create the white background
	white_bg = np.ones_like(image, dtype=np.uint8) * 255

	# Blend the original image and the white background using the alpha mask.
	# Where alpha is 1.0 (center of mask), we get the original image.
	# Where alpha is 0.0 (far from mask), we get the white background.
	# Where alpha is between 0 and 1 (the feathered edge), we get a smooth blend.
	foreground = image.astype(float)
	background = white_bg.astype(float)
	blended_image = (foreground * alpha_3ch + background * (1 - alpha_3ch)).astype(np.uint8)

	# Save the feathered masked image
	output_path = os.path.join(output_dir, file_name + ".png")
	cv2.imwrite(output_path, blended_image)

print(f"Feathered images saved to '{output_dir}' directory.")

Feathered images saved to 'masked_male_feathered' directory.


In [4]:
with open("female_annotations.json", "r") as f:
	coco = json.load(f)

images = coco['images']
annotations = coco['annotations']

ann_by_image = defaultdict(list)
for ann in annotations:
	ann_by_image[ann['image_id']].append(ann)

# Create output directory if it doesn't exist
output_dir = "masked_female_feathered"
os.makedirs(output_dir, exist_ok=True)


for img_info in images:
	file_name = img_info["file_name"].split(".")[0]
	file_name = ".".join(file_name.rsplit("_",1))
	img_id = img_info["id"]
	h, w = img_info["height"], img_info["width"]

	# Construct the image path correctly from the original file_name
	img_path = os.path.join("female", file_name)
	image = cv2.imread(img_path)

	if image is None:
		print(f"Warning: Could not read image at {img_path}")
		continue

	# Create blank mask
	mask = np.zeros((h, w), dtype=np.uint8)

	for ann in ann_by_image[img_id]:
		# Check if segmentation exists and is not empty
		if 'segmentation' in ann and ann['segmentation']:
			segmentation = ann['segmentation'][0]
			if segmentation: # Ensure the polygon is not empty
				pts = np.array(segmentation, dtype=np.int32).reshape((-1, 2))
				cv2.fillPoly(mask, [pts], color=255)

	# Apply a Gaussian blur to the binary mask. This softens the edges,
	# turning them from sharp white to a smooth gray gradient.
	# You can adjust the kernel size (e.g., (21, 21)) to increase/decrease the feathering amount.
	# The kernel size must be a pair of odd numbers.
	blur_kernel_size = (21, 21)
	blurred_mask = cv2.GaussianBlur(mask, blur_kernel_size, 0)

	# Normalize the blurred mask to be float values between 0.0 and 1.0.
	# This will act as our alpha channel.
	alpha = blurred_mask.astype(float) / 255.0
	alpha_3ch = cv2.merge([alpha, alpha, alpha])

	# Create the white background
	white_bg = np.ones_like(image, dtype=np.uint8) * 255

	# Blend the original image and the white background using the alpha mask.
	# Where alpha is 1.0 (center of mask), we get the original image.
	# Where alpha is 0.0 (far from mask), we get the white background.
	# Where alpha is between 0 and 1 (the feathered edge), we get a smooth blend.
	foreground = image.astype(float)
	background = white_bg.astype(float)
	blended_image = (foreground * alpha_3ch + background * (1 - alpha_3ch)).astype(np.uint8)

	# Save the feathered masked image
	output_path = os.path.join(output_dir, file_name + ".png")
	cv2.imwrite(output_path, blended_image)

print(f"Feathered images saved to '{output_dir}' directory.")

Feathered images saved to 'masked_female_feathered' directory.
